<a href="https://colab.research.google.com/github/sayanarajasekhar/PyTorch/blob/main/02_PyTorch_Neural_Network_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02. PyTorch Neural Network Classification

## What is a classification problem?

A classification problem involves predicting whether something is one thing or another.

| Problem type | What is it? | Example |
|--------------|-------------|---------|
| Binary classification	| Target can be one of two options,<br> e.g. yes or no | Predict whether or not someone has heart disease <br>based on their health parameters. |
|Multi-class classification	| Target can be one of more than <br>two options | 	Decide whether a photo is of food, a person or a dog. |
| Multi-label classification	| Target can be assigned <br> more than one option	| Predict what categories should be assigned to a <br>Wikipedia article (e.g. mathematics, science & philosophy). |

| Topic |	Contents |
|-------|----------|
| **0. Architecture of a <br>classification neural network**	| Neural networks can come in almost any shape or size, <br>but they typically follow a similar floor plan. |
| **1. Getting binary <br>classification data ready**	| Data can be almost anything but to get started we're going to <br>create a simple binary classification dataset. |
| **2. Building a PyTorch <br>classification model**	| Create a model to learn patterns in the data, we'll also choose a <br>**loss function**, **optimizer** and build a **training loop** specific to classification. |
| **3. Fitting the model to <br>data (training)** | 	We've got data and a model, <br>now let's let the model (try to) find patterns in the (training) data. |
| **4. Making predictions and <br>evaluating a model (inference)**	| Our model's found patterns in the data, <br>let's compare its findings to the actual (**testing**) data. |
| **5. Improving a model <br>(from a model perspective)** | We've trained and evaluated a model but it's not working, <br>let's try a few things to improve it. |
| **6. Non-linearity** | So far our model has only had the ability to model straight lines, <br>what about non-linear (non-straight) lines? |
| **7. Replicating non-linear <br>functions**	| We used **non-linear functions** to help model <br>non-linear data, but what do these look like? |
| **8. Putting it all together <br>with multi-class classification** | Let's put everything we've done so far for binary classification <br>together with a multi-class classification problem. |

## 0. Architecture of a classification neural network

Let's look at the general architecture of a classification neural network.

| Hyperparameter	| Binary Classification	| Multiclass classification |
|--------------|--------------|-----------|
| **Input layer shape** <br>(in_features) | Same as number of features (e.g. 5 for age, sex, height, <br>weight, smoking status in heart disease prediction) | Same as binary classification |
| **Hidden layer(s)** |	Problem specific, minimum = 1, maximum = unlimited	| Same as binary classification |
| **Neurons per hidden layer** | Problem specific, generally 10 to 512 | Same as binary classification |
| **Output layer shape** <br> (out_features) | 1 (one class or the other) |	1 per class (e.g. 3 for food,<br> person or dog photo) |
| **Hidden layer activation** |	Usually ReLU (rectified linear unit) <br> but can be [many others](https://en.wikipedia.org/wiki/Activation_function#Table_of_activation_functions) | Same as binary classification |
| ** Output activation** | [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) ([`torch.sigmoid`](https://pytorch.org/docs/stable/generated/torch.sigmoid.html) in PyTorch) | [Softmax](https://en.wikipedia.org/wiki/Softmax_function) ([`torch.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) in PyTorch) |
| ** Loss function** | [Binary crossentropy](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression) ([`torch.nn.BCELoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) <br>in PyTorch) | Cross entropy ([`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) <br>in PyTorch) |
| **Optimizer** | [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) (stochastic gradient descent), <br>[Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) (see [`torch.optim`](https://pytorch.org/docs/stable/optim.html) for more options)	| Same as binary classification |